In [ ]:
import sys
import os

# Adiciona o diretório raiz do projeto ao sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from config import settings
import google.generativeai as genai
from typing import Mapping, List, Tuple
from dotenv import load_dotenv
import time
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.representation import BaseRepresentation
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import pandas as pd
import random
from transformers import pipeline
import emoji
import spacy
from scipy.sparse import csr_matrix
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') # Baixar se for a primeira vez
stop_words_pt = stopwords.words('portuguese')

load_dotenv()

pd.set_option('display.float_format', '{:.2f}'.format)
API_KEY = os.getenv("API_GEMINI")

c:\Projetos\Tecnicas de NLP em dados do Instagram\av_nlp_insta\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_comments = pd.read_excel(settings.ALL_XLSX, sheet_name="reels_latestComments")

# Análise de Sentimentos

In [ ]:
# 1. Inicializa o pipeline (seu código original está perfeito)
# O modelo 'cardiffnlp/twitter-xlm-roberta-base-sentiment' retorna os rótulos: 'Positive', 'Negative', 'Neutral'
analisador_sentimento = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-xlm-roberta-base-sentiment"
)

# 2. Modifica a função para retornar tanto o LABEL quanto o SCORE
def analisar_sentimento_completo(texto):
    """
    Esta função recebe um texto, analisa o sentimento e retorna um
    pd.Series com o rótulo e a pontuação, pronto para ser adicionado
    a um DataFrame.
    """
    if pd.notna(texto) and texto.strip() != "":
        try:
            # O resultado é uma lista com um dicionário, ex: [{'label': 'Positive', 'score': 0.99}]
            resultado = analisador_sentimento(texto, truncation=True, max_length=512)[0]
            label = resultado['label']
            score = resultado['score']
            return pd.Series([label, score])
        except Exception as e:
            # Retorna None em caso de erro na análise
            return pd.Series([None, None])
    else: 
        # Retorna None se o texto for nulo ou vazio
        return pd.Series([None, None])

# 3. Aplica a nova função e cria DUAS novas colunas de uma vez
# O 'apply' vai expandir o pd.Series retornado para as colunas especificadas
df_comments[['sentiment_label', 'sentiment_score']] = df_comments['text'].apply(analisar_sentimento_completo)

# 4. Verifique o resultado
print("DataFrame com as novas colunas:")
print(df_comments[['text', 'sentiment_label', 'sentiment_score']].head())

print("\nDistribuição dos sentimentos:")
df_comments['sentiment_label'].value_counts()

# Pré-processamento

In [ ]:
# Função para remover stop words de um texto
def remover_stopwords(texto):
    if not isinstance(texto, str):  # Se não for string, retorna vazio
        return ""
    palavras = texto.split()
    palavras_filtradas = [p for p in palavras if p.lower() not in stop_words_pt]
    return " ".join(palavras_filtradas)

# Aplicar a todos os comentários negativos
df_comments['text clean'] = df_comments['text'].apply(remover_stopwords)

# Modelagem de Tópicos

In [ ]:
class GeminiDocsRefiner(BaseRepresentation):
    def __init__(self, api_key: str, model: str = "gemini-2.0-flash", prompt_template: str = None):
        genai.configure(api_key=api_key)
        self.model = model
        # Prompt aprimorado para receber exemplos de documentos
        self.prompt_template = prompt_template or (
            "Escreva uma descrição de um parágrafo que descreva detalhadamente o que os comentários do instagram presentes neste tópico tem em comum: {documents}"
        )

    def extract_topics(
        self,
        topic_model,
        documents: pd.DataFrame, # Recebe o DataFrame de documentos
        c_tf_idf: csr_matrix,
        topics: Mapping[str, List[Tuple[str, float]]]
    ) -> Mapping[str, List[Tuple[str, float]]]:
        
        updated_topics = {}
        # Mapeia cada documento ao seu tópico para fácil acesso
        docs_per_topic = documents.groupby(['Topic'])['Document'].apply(list)

        for topic_id, keywords in topics.items():
            
            if topic_id == -1:
                updated_topics[topic_id] = keywords
                continue

            if topic_id % 10 == 0:
                time.sleep(60)
            
            try:
                # Pega os documentos reais do tópico
                topic_docs = docs_per_topic.get(topic_id, [])
                if not topic_docs:
                    updated_topics[topic_id] = keywords
                    continue

                # Pega uma amostra para não exceder o limite do prompt
                sample_size = min(len(topic_docs), 10) # Amostra de até 10 documentos
                docs_sample = "\n- ".join(random.sample(topic_docs, sample_size))
                
                prompt = self.prompt_template.format(documents=docs_sample)

                response = genai.GenerativeModel(self.model).generate_content(prompt)
                label = response.text.strip() if hasattr(response, "text") else None

                if label:
                    # Adiciona o novo rótulo com a maior pontuação
                    updated_keywords = [(label, 1.0)] + keywords 
                    updated_topics[topic_id] = updated_keywords
                else:
                    updated_topics[topic_id] = keywords  # Fallback

            except Exception as e:
                print(f"[GeminiDocsRefiner] Erro no tópico {topic_id}: {e}")
                updated_topics[topic_id] = keywords  # Fallback

        return updated_topics

In [ ]:
list_dataframes = []

representation_model = GeminiDocsRefiner(api_key=API_KEY)

# Regex: captura palavras com 2+ caracteres OU qualquer caractere no bloco de Emojis Unicode
# Isso mantém as palavras normais e adiciona os emojis como tokens
emoji_pattern = r"(?u)\b\w\w+\b|[\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\u2600-\u26FF\u2700-\u27BF]"

vectorizer_model = CountVectorizer(token_pattern=emoji_pattern)

topic_model = BERTopic(
    vectorizer_model=vectorizer_model,
    representation_model=representation_model,
    embedding_model="rufimelo/bert-large-portuguese-cased-sts",
    language="multilingual",
    verbose=True
)

# Supondo que df_comments é seu DataFrame
# A função demojize transforma '😂' em ':face_with_tears_of_joy:'
# Usar language='pt' pode ajudar a obter descrições em português, se disponíveis.
def demojize_text(text):
    return emoji.demojize(text, language='pt')

df_comments['text_demojized'] = df_comments['text clean'].apply(demojize_text)

# Agora, use esta nova coluna para a modelagem
docs_completos = list(df_comments['text_demojized'])

print(f"Iniciando a modelagem de tópicos em {len(docs_completos)} documentos...")
topics, probs = topic_model.fit_transform(docs_completos)
print("Modelagem global de tópicos concluída.")

# Obtém as informações de tópico para TODOS os documentos
topic_info_df = topic_model.get_document_info(docs_completos)

# Reseta o índice de ambos para garantir o alinhamento
df_comments_reset = df_comments.reset_index(drop=True)
topic_info_df_reset = topic_info_df.reset_index(drop=True)

# Concatena os DataFrames originais com os resultados da modelagem
df_final = pd.concat([df_comments_reset, topic_info_df_reset], axis=1)

# (Opcional) Remove a coluna de documento duplicada
df_final = df_final.drop(columns=['Document'])

print("DataFrame final com resultados globais:")
df_final.head()

# Salvar dados Modelados

In [7]:
try:
    # 2. Use o pd.ExcelWriter em um bloco 'with' para garantir que o arquivo seja salvo e fechado corretamente
    with pd.ExcelWriter(
        settings.ALL_XLSX,
        mode="a",  # 'a' significa "append" (adicionar/anexar)
        engine="openpyxl",
        if_sheet_exists="replace"  # Substitui a aba se ela já existir
    ) as writer:
        
        # 3. Salve seu DataFrame na aba especificada
        df_final.to_excel(writer, sheet_name='reels_latestComments', index=False)
        # 'index=False' é importante para não salvar o índice do DataFrame como uma coluna no Excel

except FileNotFoundError:
    print(f"Aviso: O arquivo '{settings.ALL_XLSX}' não foi encontrado.")
    print("Um novo arquivo será criado com a aba especificada.")
    # Se o arquivo não existir, o modo 'a' falha. Então, simplesmente o criamos.
    df_final.to_excel('reels_latestComments.xlsx', sheet_name='reels_latestComments', index=False)

except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")